In [1]:
import pymysql
import numpy as np
import pandas as pd

In [2]:
def get_mean_taste():
    conn = pymysql.connect(host = "localhost",
                user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()
    cursor.execute("select menu_id, restaurant_id,AVG(flavor) from young_cheline.evaluate group by menu_id, restaurant_id;")
    res = cursor.fetchall()
    return res

In [3]:
def get_count():
    conn = pymysql.connect(host = "localhost",
                user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()
    cursor.execute("select count(*) from young_cheline.evaluate group by menu_id;")
    g_menu_id = cursor.fetchall()

    cursor.execute('''SELECT menu_id, COUNT(CASE WHEN couple LIKE '0' THEN 1 END) AS specific_count
        FROM young_cheline.evaluate
        GROUP BY menu_id;''')
    
    couple = cursor.fetchall()
    
    cursor.execute('''SELECT menu_id, COUNT(CASE WHEN family LIKE '1' THEN 1 END) AS specific_count
        FROM young_cheline.evaluate
        GROUP BY menu_id;''')
    
    family = cursor.fetchall()

    cursor.execute('''SELECT menu_id, COUNT(CASE WHEN solo LIKE '2' THEN 1 END) AS specific_count
        FROM young_cheline.evaluate
        GROUP BY menu_id;''')
    
    solo = cursor.fetchall()

    cursor.execute('''SELECT menu_id, COUNT(CASE WHEN drink LIKE '3' THEN 1 END) AS specific_count
                    FROM young_cheline.evaluate
                    GROUP BY menu_id;''')
    
    drink = cursor.fetchall()

    cursor.execute('''SELECT menu_id, COUNT(CASE WHEN friend LIKE '4' THEN 1 END) AS specific_count
        FROM young_cheline.evaluate
        GROUP BY menu_id;''')
    
    friend = cursor.fetchall()

    cursor.execute('''SELECT menu_id, AVG(NULLIF(cleaning, '')),AVG(NULLIF(plating, '')),AVG(NULLIF(price, '')),AVG(NULLIF(service, ''))
        FROM young_cheline.evaluate
        GROUP BY menu_id;''')
    
    avg_exxtra = cursor.fetchall()

    cursor.execute('''SELECT menu_name FROM young_cheline.menu;''')
    menu_name = cursor.fetchall()

    eva = get_mean_taste()

    for menu_id in range(len(g_menu_id)):
        menu_count =g_menu_id[menu_id][0] #메뉴별 평가 갯수
        
        menu = eva[menu_id][0]
        restaurant_id = eva[menu_id][1]
        avg_flavor = eva[menu_id][2]
        avg_flavor = round(avg_flavor)
        in_menu_name = menu_name[menu_id][0]

        cleaning = avg_exxtra[menu_id][1]
        cleaning = round(cleaning) if isinstance(cleaning, float) else cleaning

        plating = avg_exxtra[menu_id][2]
        plating = round(cleaning) if isinstance(cleaning, float) else cleaning

        price = avg_exxtra[menu_id][3]
        price = round(cleaning) if isinstance(cleaning, float) else cleaning
        
        service = avg_exxtra[menu_id][4]
        service = round(cleaning) if isinstance(cleaning, float) else cleaning

        ## mood
        solo2 = 2 if  (solo[menu_id][1]/ menu_count) >0.8 else None
        couple0 = 0 if  (couple[menu_id][1]/ menu_count) >0.8 else None
        drink3 = 3 if  (drink[menu_id][1]/ menu_count) >0.8 else None
        family1 = 1 if  (family[menu_id][1]/ menu_count) >0.8 else None
        friend4 = 4 if  (friend[menu_id][1]/ menu_count) >0.8 else None

        cursor.execute('''INSERT INTO young_cheline.restaurant_evaluate
            (menu_id,menu_name, cleaning, plating, price, restaurant_id, service, flavor, couple, drink, family, friend, solo)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''',
                       (menu, in_menu_name,cleaning, plating, price, restaurant_id, service, avg_flavor, couple0, drink3, family1, friend4, solo2))

    
    conn.commit()


In [4]:
if __name__ == "__main__":
    res = get_count()

In [5]:
import pymysql
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

In [7]:
def get_data():
    conn = pymysql.connect(host = "localhost",
                user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()
    cursor.execute("SELECT flavor, menu_id, user_name FROM young_cheline.evaluate;")
    res = cursor.fetchall()
    return res

In [8]:
def make_ptable(df):
    ptable = df.pivot_table(values = 'flavor', index = 'menu_id', columns = 'user_name') 
    
    return ptable

In [9]:
def cosine(ptable):
    ptable = ptable.fillna(0)
    menu_id = ptable.index

    cos = cosine_similarity(ptable)
    cos = pd.DataFrame(data = cos, index=menu_id, columns=menu_id)
    return cos

In [10]:
def search_cos(cos,pt, user_id: str):
    ex = pt.assign(menu=pt.index.to_list())

    pt=ex.loc[ex[user_id].notnull(), [user_id, 'menu']] 
    user_cos=cos[pt['menu']] ## cos 확인 
    user_cos=user_cos[~user_cos.isin(user_cos.loc[pt['menu']])].dropna()
    X_vector = pt.iloc[:,[0]].values/10+1
    Y_matrix= user_cos.to_numpy()
    Matrix_product=Y_matrix.dot(X_vector)
    user_cos['Sum']=Matrix_product
    user_recomand=user_cos.sort_values('Sum',ascending=False).head(10).index.to_list()
    return user_recomand

In [11]:
def data_open():
    data = get_data()
    df = pd.DataFrame(data, columns=['flavor', 'menu_id','user_name'])
    df['flavor'] = df['flavor'].astype(int)
    return df

In [12]:
def recommand(user_id):
    df=data_open()
    ptable = make_ptable(df)
    cos = cosine(ptable)
    a = search_cos(cos,ptable, user_id=user_id)
    print(user_id,a)
    return user_id, a

In [15]:
def insert_db(list):
    conn = pymysql.connect(host = "localhost",
                user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()

    for a in list:
        menu_id = a[0]
        user_name = a[1]
        current_time = datetime.now()
        cursor.execute('''INSERT INTO young_cheline.recommend (menu_id,user_name,time)
                        VALUES (%s, %s,%s);''',(menu_id, user_name,current_time))
    conn.commit()

In [16]:
if __name__ == '__main__':
    user, reco=recommand('ehddbs4521')
    list=[]
    us = 'ehddbs4521'
    for i in reco:
        list.append([i,us])

    insert_db(list)

ehddbs4521 [33, 26, 30, 7, 21, 16, 29, 15, 14, 19]
